# Item relation matrices

### Get the data

In [3]:
import polars as pl

# define the schema of the dataframe
event_schema = pl.Struct({"aid": pl.UInt32, "ts": pl.UInt64, "type": str})
df_schema = {"session": pl.UInt32, "events": pl.List(event_schema)}

df = pl.read_ndjson('../data/train.jsonl', schema=df_schema, low_memory=True)

print(df)

shape: (12_899_779, 2)
┌──────────┬─────────────────────────────────┐
│ session  ┆ events                          │
│ ---      ┆ ---                             │
│ u32      ┆ list[struct[3]]                 │
╞══════════╪═════════════════════════════════╡
│ 0        ┆ [{1517085,1659304800025,"click… │
│ 1        ┆ [{424964,1659304800025,"carts"… │
│ 2        ┆ [{763743,1659304800038,"clicks… │
│ 3        ┆ [{1425967,1659304800095,"carts… │
│ 4        ┆ [{613619,1659304800119,"clicks… │
│ …        ┆ …                               │
│ 12899774 ┆ [{33035,1661723968869,"clicks"… │
│ 12899775 ┆ [{1743151,1661723970935,"click… │
│ 12899776 ┆ [{548599,1661723972537,"clicks… │
│ 12899777 ┆ [{384045,1661723976974,"clicks… │
│ 12899778 ┆ [{561560,1661723983611,"clicks… │
└──────────┴─────────────────────────────────┘


In [4]:
# Sessions
exploded_df = (
    df
    .explode("events")
    .unnest("events")
)

print(exploded_df)

shape: (216_716_096, 4)
┌──────────┬─────────┬───────────────┬────────┐
│ session  ┆ aid     ┆ ts            ┆ type   │
│ ---      ┆ ---     ┆ ---           ┆ ---    │
│ u32      ┆ u32     ┆ u64           ┆ str    │
╞══════════╪═════════╪═══════════════╪════════╡
│ 0        ┆ 1517085 ┆ 1659304800025 ┆ clicks │
│ 0        ┆ 1563459 ┆ 1659304904511 ┆ clicks │
│ 0        ┆ 1309446 ┆ 1659367439426 ┆ clicks │
│ 0        ┆ 16246   ┆ 1659367719997 ┆ clicks │
│ 0        ┆ 1781822 ┆ 1659367871344 ┆ clicks │
│ …        ┆ …       ┆ …             ┆ …      │
│ 12899776 ┆ 1737908 ┆ 1661723987073 ┆ clicks │
│ 12899777 ┆ 384045  ┆ 1661723976974 ┆ clicks │
│ 12899777 ┆ 384045  ┆ 1661723986800 ┆ clicks │
│ 12899778 ┆ 561560  ┆ 1661723983611 ┆ clicks │
│ 12899778 ┆ 32070   ┆ 1661723994936 ┆ clicks │
└──────────┴─────────┴───────────────┴────────┘


In [5]:
def get_sub_sessions(only_clicks=False, with_row_index=False, limit=None):
    """
    Splits sessions into sub sessions based on time between events or session boundaries.

    Args:
        only_clicks: only clicks are considered in the sub sessions
        with_row_index: add row index to the sub sessions in order to calculate delta time between events
        limit: limit the amount of rows to be processed.

    Returns: DataFrame of sub sessions
    """

    sub_sessions = exploded_df

    if limit is not None:
        sub_sessions = exploded_df.limit(limit)
    if only_clicks:
        sub_sessions = sub_sessions.filter(pl.col("type") == "clicks")

    sub_sessions = (
        sub_sessions
        # Convert ts to seconds and cast to UInt32 to save memory
        .with_columns((pl.col("ts")//1000).cast(pl.UInt32))
        .sort(["session", "ts"])
        .with_columns(
            next_session = pl.col("session").shift(-1),
            next_aid = pl.col("aid").shift(-1),
            next_ts = pl.col("ts").shift(-1),
            next_type = pl.col("type").shift(-1),
        )
        # Row is a sub session boundary if there is existing session boundary or if time between events is more than 30 minutes
        .with_columns(
            is_session_boundary = ((pl.col("session") != pl.col("next_session")) | (pl.col("next_ts") - pl.col("ts") > 1800)),
        )
        .with_columns(
            sub_session = pl.col("is_session_boundary").cum_sum().cast(pl.UInt32),
        )
        # Filter out session boundaries. This also removes sub sessions with only 1 event which are not interesting
        .filter(pl.col("is_session_boundary").not_())
        .drop("is_session_boundary")
    )

    if with_row_index:
        sub_sessions = sub_sessions.with_row_index()

    return sub_sessions

# sub_sessions = get_sub_sessions()
# print(sub_sessions)
# print("Amount of sub sessions:", sub_sessions.select("sub_session").n_unique())

In [6]:
# Helper functions

def get_clicks_of_sub_session(sub_sessions):
    return (
        sub_sessions
        .filter(pl.col("type") == "clicks")
        .select(["sub_session", "ts", "aid"])
        .rename({"ts": "click_ts", "aid": "click_aid"})
    )

def get_carts_of_sub_session(sub_sessions):
    return (
        sub_sessions
        .filter(pl.col("type") == "carts")
        .select(["sub_session", "ts", "aid"])
        .rename({"ts": "cart_ts", "aid": "cart_aid"})
    )

def get_orders_of_sub_session(sub_sessions):
    return (
        sub_sessions
        .filter(pl.col("type") == "orders")
        .select(["sub_session", "ts", "aid"])
        .rename({"ts": "order_ts", "aid": "order_aid"})
    )

### Click to click matrix
Click to click matrix is defined as the probabilities of other aids being clicked after the previous aid is clicked.
Click to click matrix is formed from the sub sessions since there is no point in counting subsequent clicks that are from a user coming back to site after a long time

We do multiple variations of the click to click matrix:

Next click only

In [ ]:
sub_sessions = get_sub_sessions()

# Count how many same click to click events there are
subsequent_clicks_count = (
    sub_sessions
    .filter((pl.col("type") == "clicks") & (pl.col("next_type") == "clicks"))
    .group_by(["aid", "next_aid"])
    .agg(pl.len().alias("count"))
)

# Sum all the clicks for each aid
aid_clicks_sum = (
    subsequent_clicks_count
    .group_by("aid")
    .agg(pl.sum("count").alias("sum"))
)

# Calculate the probabilities of items being clicked immediately after another item has been clicked
click_to_click_matrix = (
    subsequent_clicks_count
    .join(aid_clicks_sum, on="aid")
    .with_columns(
        probability = pl.col("count") / pl.col("sum")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "sum"])
    .sort(["aid", "next_aid"])
)

print(click_to_click_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", click_to_click_matrix.select("aid").n_unique())
print("Total probability:", click_to_click_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
click_to_click_matrix.write_csv("./click_to_click_matrix.csv")

Next click only. Remove carts and orders from the sub sessions.

In [ ]:
sub_sessions = get_sub_sessions(only_clicks=True)

# Count how many same click to click events there are
subsequent_clicks_count = (
    sub_sessions
    .group_by(["aid", "next_aid"])
    .agg(pl.len().alias("count"))
)

# Sum all the clicks for each aid
aid_clicks_sum = (
    subsequent_clicks_count
    .group_by("aid")
    .agg(pl.sum("count").alias("sum"))
)

# Calculate the probabilities of items being clicked immediately after another item has been clicked
click_to_click_matrix = (
    subsequent_clicks_count
    .join(aid_clicks_sum, on="aid")
    .with_columns(
        probability = pl.col("count") / pl.col("sum")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "sum"])
    .sort(["aid", "next_aid"])
)

print(click_to_click_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", click_to_click_matrix.select("aid").n_unique())
print("Total probability:", click_to_click_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
click_to_click_matrix.write_csv("./click_to_click_matrix_only_clicks.csv")

### Click to cart matrix
Click to cart matrix is defined as the probabilities of other aids being added to cart in the same sub session after an aid is clicked.
Click to cart matrix is formed from the sub sessions since the sub session should show clear intent of the user to buy items they click.

In [ ]:
# Get clicks and carts of sub sessions
clicks_of_sub_sessions = get_clicks_of_sub_session(sub_sessions)

carts_of_sub_sessions = get_carts_of_sub_session(sub_sessions)

carts_after_clicks_in_sub_sessions = (
    carts_of_sub_sessions
    # Combine clicks and carts of sub sessions
    .join(clicks_of_sub_sessions, on="sub_session", how="inner")
    # Keep only carts that happened after clicks
    .filter(pl.col("click_ts") < pl.col("cart_ts"))
    .select(["sub_session", "click_aid", "cart_aid"])
)

# Count how many same click to cart events there are
click_to_cart_count = (
    carts_after_clicks_in_sub_sessions
    .group_by(["click_aid", "cart_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the clicks for each aid
aid_clicks_sum = (
    click_to_cart_count
    .group_by("click_aid")
    .agg(pl.sum("count").alias("sum"))
    .sort("sum", descending=True)
)

# Calculate the probabilities of items being added to cart after another item has been clicked
click_to_cart_matrix = (
    click_to_cart_count
    .join(aid_clicks_sum, on="click_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("sum")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "sum"])
    .sort(["click_aid", "cart_aid"])
)

print(click_to_cart_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", click_to_cart_matrix.select("click_aid").n_unique())
print("Total probability:", click_to_cart_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
click_to_cart_matrix.write_csv("./click_to_cart_matrix.csv")

### Click to order matrix
Click to order matrix is defined as the probabilities of other aids being ordered in the same sub session after an aid is clicked.
Click to order matrix is formed from the sub sessions since the sub session should show clear intent of the user to buy items they click.

In [ ]:
# Get clicks and orders of sub sessions
clicks_of_sub_sessions = get_clicks_of_sub_session(sub_sessions)

orders_of_sub_sessions = get_orders_of_sub_session(sub_sessions)

orders_after_clicks_in_sub_sessions = (
    orders_of_sub_sessions
    # Combine clicks and orders of sub sessions
    .join(clicks_of_sub_sessions, on="sub_session", how="inner")
    # Keep only orders that happened after clicks
    .filter(pl.col("click_ts") < pl.col("order_ts"))
    .select(["sub_session", "click_aid", "order_aid"])
)

# Count how many same click to order events there are
click_to_order_count = (
    orders_after_clicks_in_sub_sessions
    .group_by(["click_aid", "order_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the clicks for each aid
aid_clicks_sum = (
    click_to_order_count
    .group_by("click_aid")
    .agg(pl.sum("count").alias("sum"))
    .sort("sum", descending=True)
)

# Calculate the probabilities of items being ordered after another item has been clicked
click_to_order_matrix = (
    click_to_order_count
    .join(aid_clicks_sum, on="click_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("sum")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "sum"])
    .sort(["click_aid", "order_aid"])
)

print(click_to_order_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", click_to_order_matrix.select("click_aid").n_unique())
print("Total probability:", click_to_order_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
click_to_order_matrix.write_csv("./click_to_order_matrix.csv")

### Cart to click matrix
Cart to click matrix is defined as the probabilities of other aids being clicked immediately after an aid is added to cart.

In [ ]:
# Get carts and clicks of sub sessions
carts_of_sub_sessions = get_carts_of_sub_session(sub_sessions)

clicks_of_sub_sessions = get_clicks_of_sub_session(sub_sessions)

clicks_after_carts_in_sub_sessions = (
    clicks_of_sub_sessions
    # Combine carts and clicks of sub sessions
    .join(carts_of_sub_sessions, on="sub_session", how="inner")
    # Keep only clicks that happened after carts
    .filter(pl.col("cart_ts") < pl.col("click_ts"))
    .select(["sub_session", "cart_aid", "click_aid"])
)

# Count how many same cart to click events there are
cart_to_click_count = (
    clicks_after_carts_in_sub_sessions
    .group_by(["cart_aid", "click_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the carts for each aid
aid_carts_sum = (
    cart_to_click_count
    .group_by("cart_aid")
    .agg(pl.sum("count").alias("sum"))
    .sort("sum", descending=True)
)

# Calculate the probabilities of items being clicked immediately after another item has been added to cart
cart_to_click_matrix = (
    cart_to_click_count
    .join(aid_carts_sum, on="cart_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("sum")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "sum"])
    .sort(["cart_aid", "click_aid"])
)

print(cart_to_click_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", cart_to_click_matrix.select("cart_aid").n_unique())
print("Total probability:", cart_to_click_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
cart_to_click_matrix.write_csv("./cart_to_click_matrix.csv")

### Cart to cart matrix
Cart to cart matrix is defined as the probabilities of other aids being added to cart later in the same sub session where an aid is added to cart.

In [ ]:
# Get carts of sub sessions
carts_of_sub_sessions = get_carts_of_sub_session(sub_sessions)

next_carts_of_sub_sessions = (
    carts_of_sub_sessions
    .rename({"cart_ts": "next_cart_ts", "cart_aid": "next_cart_aid"})
)

# Find subsequent carts in the same sub session
subsequent_carts = (
    carts_of_sub_sessions
    .join(next_carts_of_sub_sessions, on="sub_session", how="inner")
    .filter(pl.col("cart_ts") < pl.col("next_cart_ts"))
    .select(["sub_session", "cart_aid", "next_cart_aid"])
)

# Count how many same cart to cart events there are
subsequent_carts_count = (
    subsequent_carts
    .group_by(["cart_aid", "next_cart_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the carts for each aid
aid_carts_sum = (
    subsequent_carts_count
    .group_by("cart_aid")
    .agg(pl.sum("count").alias("sum"))
    .sort("sum", descending=True)
)

# Calculate the probabilities of items being added to cart after another item has been added to cart
cart_to_cart_matrix = (
    subsequent_carts_count
    .join(aid_carts_sum, on="cart_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("sum")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "sum"])
    .sort(["cart_aid", "next_cart_aid"])
)

print(cart_to_cart_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", cart_to_cart_matrix.select("aid").n_unique())
print("Total probability:", cart_to_cart_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
cart_to_cart_matrix.write_csv("./cart_to_cart_matrix.csv")

### Cart to order matrix
Cart to order matrix is defined as the probabilities of other aids being ordered later in the same sub session where an aid is added to cart.

In [ ]:
# Get carts and orders of sub sessions
carts_of_sub_sessions = get_carts_of_sub_session(sub_sessions)

orders_of_sub_sessions = get_orders_of_sub_session(sub_sessions)

orders_after_carts_in_sub_sessions = (
    orders_of_sub_sessions
    # Combine carts and orders of sub sessions
    .join(carts_of_sub_sessions, on="sub_session", how="inner")
    # Keep only orders that happened after carts
    .filter(pl.col("cart_ts") < pl.col("order_ts"))
    .select(["sub_session", "cart_aid", "order_aid"])
)

# Count how many same cart to order events there are
cart_to_order_count = (
    orders_after_carts_in_sub_sessions
    .group_by(["cart_aid", "order_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the carts for each aid
aid_carts_sum = (
    cart_to_order_count
    .group_by("cart_aid")
    .agg(pl.sum("count").alias("sum"))
    .sort("sum", descending=True)
)

# Calculate the probabilities of items being ordered after another item has been added to cart
cart_to_order_matrix = (
    cart_to_order_count
    .join(aid_carts_sum, on="cart_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("sum")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "sum"])
    .sort(["cart_aid", "order_aid"])
)

print(cart_to_order_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", cart_to_order_matrix.select("cart_aid").n_unique())
print("Total probability:", cart_to_order_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
cart_to_order_matrix.write_csv("./cart_to_order_matrix.csv")

### Order to click matrix
Order to click matrix is defined as the probabilities of other aids being clicked immediately after an aid is ordered.

In [ ]:
# Get orders and clicks of sub sessions
orders_of_sub_sessions = get_orders_of_sub_session(sub_sessions)

clicks_of_sub_sessions = get_clicks_of_sub_session(sub_sessions)

clicks_after_orders_in_sub_sessions = (
    clicks_of_sub_sessions
    # Combine orders and clicks of sub sessions
    .join(orders_of_sub_sessions, on="sub_session", how="inner")
    # Keep only clicks that happened after orders
    .filter(pl.col("order_ts") < pl.col("click_ts"))
    .select(["sub_session", "order_aid", "click_aid"])
)

# Count how many same order to click events there are
order_to_click_count = (
    clicks_after_orders_in_sub_sessions
    .group_by(["order_aid", "click_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the orders for each aid
aid_orders_sum = (
    order_to_click_count
    .group_by("order_aid")
    .agg(pl.sum("count").alias("sum"))
    .sort("sum", descending=True)
)

# Calculate the probabilities of items being clicked immediately after another item has been ordered
order_to_click_matrix = (
    order_to_click_count
    .join(aid_orders_sum, on="order_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("sum")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "sum"])
    .sort(["order_aid", "click_aid"])
)

print(order_to_click_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", order_to_click_matrix.select("order_aid").n_unique())
print("Total probability:", order_to_click_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
order_to_click_matrix.write_csv("./order_to_click_matrix.csv")

### Order to cart matrix
Order to cart matrix is defined as the probabilities of other aids being added to cart later in the same sub session where an aid is ordered.

In [ ]:
# Get orders and carts of sub sessions
orders_of_sub_sessions = get_orders_of_sub_session(sub_sessions)

carts_of_sub_sessions = get_carts_of_sub_session(sub_sessions)

carts_after_orders_in_sub_sessions = (
    carts_of_sub_sessions
    # Combine orders and carts of sub sessions
    .join(orders_of_sub_sessions, on="sub_session", how="inner")
    # Keep only carts that happened after orders
    .filter(pl.col("order_ts") < pl.col("cart_ts"))
    .select(["sub_session", "order_aid", "cart_aid"])
)

# Count how many same order to cart events there are
order_to_cart_count = (
    carts_after_orders_in_sub_sessions
    .group_by(["order_aid", "cart_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the orders for each aid
aid_orders_sum = (
    order_to_cart_count
    .group_by("order_aid")
    .agg(pl.sum("count").alias("sum"))
    .sort("sum", descending=True)
)

# Calculate the probabilities of items being added to cart after another item has been ordered
order_to_cart_matrix = (
    order_to_cart_count
    .join(aid_orders_sum, on="order_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("sum")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "sum"])
    .sort(["order_aid", "cart_aid"])
)

print(order_to_cart_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", order_to_cart_matrix.select("order_aid").n_unique())
print("Total probability:", order_to_cart_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
order_to_cart_matrix.write_csv("./order_to_cart_matrix.csv")

### Order to order matrix
Order to order matrix is defined as the probabilities of other aids being ordered later in the same sub session where an aid is ordered.

In [ ]:
# Get orders of sub sessions
orders_of_sub_sessions = get_orders_of_sub_session(sub_sessions)

next_orders_of_sub_sessions = (
    orders_of_sub_sessions
    .rename({"order_ts": "next_order_ts", "order_aid": "next_order_aid"})
)

# Find subsequent orders in the same sub session
subsequent_orders = (
    orders_of_sub_sessions
    .join(next_orders_of_sub_sessions, on="sub_session", how="inner")
    .filter(pl.col("order_ts") < pl.col("next_order_ts"))
    .select(["sub_session", "order_aid", "next_order_aid"])
)

# Count how many same order to order events there are
subsequent_orders_count = (
    subsequent_orders
    .group_by(["order_aid", "next_order_aid"])
    .agg(pl.len().alias("count"))
    .sort("count", descending=True)
)

# Sum all the orders for each aid
aid_orders_sum = (
    subsequent_orders_count
    .group_by("order_aid")
    .agg(pl.sum("count").alias("sum"))
    .sort("sum", descending=True)
)

# Calculate the probabilities of items being ordered after another item has been ordered
order_to_order_matrix = (
    subsequent_orders_count
    .join(aid_orders_sum, on="order_aid")
    .with_columns(
        probability = pl.col("count") / pl.col("sum")
    )
    .with_columns(pl.col("probability").cast(pl.Float32))
    .drop(["count", "sum"])
    .sort(["order_aid", "next_order_aid"])
)

print(order_to_order_matrix)

# Check that probabilities sum to the amount of unique aids
print("Unique aids:", order_to_order_matrix.select("order_aid").n_unique())
print("Total probability:", order_to_order_matrix.select("probability").sum().select(pl.first()).item())

In [ ]:
# save to csv
order_to_order_matrix.write_csv("./order_to_order_matrix.csv")

### Order incompatibility matrix
Order incompatibility matrix is defined as links between items which are often ordered with same items but are never ordered together.

In [21]:
sub_sessions = get_sub_sessions()

orders_of_sub_sessions_df = get_orders_of_sub_session(sub_sessions)

# how many times item has to be ordered with another item so that their relation can be considered strong
ordered_together_threshold = 4

allowed_aids_df = (
    orders_of_sub_sessions_df
    .group_by("order_aid")
    .agg(pl.len().alias("count"))
    .filter(pl.col("count") >= ordered_together_threshold)
    .sort("order_aid")
)

print(allowed_aids_df)

multi_order_sub_sessions_df = (
    orders_of_sub_sessions_df
    # filter aids with too few orders
    .join(allowed_aids_df.select("order_aid"), on="order_aid", how="inner")
    # group orders and get unique aids for each order
    .group_by("sub_session")
    # Remove items ordered multiple times in single session
    .agg(pl.col("order_aid").unique().sort())
    # filter out orders with only one item left
    .filter(pl.col("order_aid").list.len() > 1)
    .explode("order_aid")
)

print(multi_order_sub_sessions_df)

bought_together_count = (
    multi_order_sub_sessions_df
    .join(multi_order_sub_sessions_df, on="sub_session", how="inner")
    .drop("sub_session")
    .group_by(["order_aid", "order_aid_right"])
    .agg(pl.len().alias("count"))
    # How many times ordered together
    .filter((pl.col("count") >= ordered_together_threshold) & (pl.col("order_aid") != pl.col("order_aid_right")))
)

print(bought_together_count)

aids_bought_together_df = (
    bought_together_count
    .group_by("order_aid")
    .agg(pl.col("order_aid_right").alias("bought_together_with"))
)

print(aids_bought_together_df)

incompatible_pairs = []

print("Start")
for index, order_aid in enumerate(aids_bought_together_df.select("order_aid").to_numpy().reshape(-1)):
    if (index+1) % 1000 == 0:
        print("row", index+1)

    aids_bought_together = aids_bought_together_df.row(by_predicate=(pl.col("order_aid") == order_aid))[1]
    for aid in aids_bought_together:
        incompatible_aids = aids_bought_together_df.row(by_predicate=(pl.col("order_aid") == aid))[1]
        new_incompatible_pairs = [(order_aid, aid) for aid in incompatible_aids]
        incompatible_pairs.extend(new_incompatible_pairs)

print("Done")
print()

incompatible_df = (
    pl.DataFrame(
        data=incompatible_pairs,
        orient='row',
        schema={"aid": pl.UInt32, "incompatible_aid": pl.UInt32}
    )
    .filter(pl.col("aid") != pl.col("incompatible_aid"))
    .unique()
    .sort(["aid", "incompatible_aid"])
)
print(incompatible_df)

shape: (145_610, 2)
┌───────────┬───────┐
│ order_aid ┆ count │
│ ---       ┆ ---   │
│ u32       ┆ u32   │
╞═══════════╪═══════╡
│ 3         ┆ 13    │
│ 21        ┆ 15    │
│ 32        ┆ 4     │
│ 39        ┆ 12    │
│ 50        ┆ 6     │
│ …         ┆ …     │
│ 1855571   ┆ 4     │
│ 1855572   ┆ 14    │
│ 1855576   ┆ 6     │
│ 1855582   ┆ 5     │
│ 1855594   ┆ 40    │
└───────────┴───────┘
shape: (1_897_899, 2)
┌─────────────┬───────────┐
│ sub_session ┆ order_aid │
│ ---         ┆ ---       │
│ u32         ┆ u32       │
╞═════════════╪═══════════╡
│ 3831086     ┆ 506170    │
│ 3831086     ┆ 866814    │
│ 3831086     ┆ 1364631   │
│ 3831086     ┆ 1409907   │
│ 3831086     ┆ 1502524   │
│ …           ┆ …         │
│ 37571301    ┆ 1716062   │
│ 16332033    ┆ 801774    │
│ 16332033    ┆ 946991    │
│ 16332033    ┆ 1012438   │
│ 16332033    ┆ 1598282   │
└─────────────┴───────────┘
shape: (118_652, 3)
┌───────────┬─────────────────┬───────┐
│ order_aid ┆ order_aid_right ┆ count │
│ ---   

In [22]:
# Write to csv
incompatible_df.write_csv("./incompatible_matrix.csv")

In [26]:
unique_incompatible_count = (
    incompatible_df
    .select("aid")
    .n_unique()
)

print(f"Incompatible products found for {unique_incompatible_count} unique products")

Incompatible products found for 16606 unique products
